In [13]:
import pandas as pd
import pandahouse as ph

In [14]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [15]:
q = '''
    SELECT
        count(distinct st_id) as qt_st,
        date_m 
    FROM
        (
        SELECT
            st_id,
            toStartOfMonth(timest) AS date_m,
            SUM(correct) AS total_correct
        FROM default.peas
        GROUP BY date_m , st_id
        HAVING total_correct >=20
        )
    GROUP BY date_m
    '''

q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,qt_st,date_m
0,136,2021-10-01


In [17]:
qqq = '''
    select 
    tabel_1.test_grp as group_,
    round(ARPU,2) as ARPU ,
    round(ARPAU,2) as ARPAU ,
    round(CR,2) as CR,
    round(CR_activ,2) as CR_activ,
    round(CR_math,2) as CR_math
from 
    (
    SELECT
        A.test_grp as test_grp,
        sum(money) / count(distinct st_id) as ARPU -- sum of revenue/ count of all users
    FROM
        default.studs as A
    LEFT JOIN
        default.final_project_check as B
    ON
        A.st_id = B.st_id
    group by test_grp
    )   
as tabel_1
left join 
    (
    select 
test_grp,
sum(sum_money)/ count(id) as ARPAU -- sum of revenue/ count of all active users
from 
    (
        select 
            st_id as st_id,
            sum(money) as sum_money
        from default.final_project_check
        group by st_id
        ) as a
    right join 
        (
        select 
            st_id as id,
            sum(correct) as sum_cor
        from default.peas
        group by st_id
        having sum_cor>10
        ) as b
    on a.st_id=b.id
    inner join 
        (
        select 
            test_grp,
            st_id
        from default.studs
        ) as c
    on b.id=c.st_id
    group by test_grp
        )
as tabel_2
on tabel_1.test_grp=tabel_2.test_grp
left join 
    (
    select test_grp,
       student_count*100/st_count as CR   -- count of buyers/ count of all users
    from (
    select test_grp ,
           count(distinct st_id) as st_count
    from default.studs
    group by test_grp) as t1
    left join (
    select test_grp,
           count( distinct st_id) as student_count
    from default.final_project_check as t_1
    join default.studs as t_2 on t_1.st_id=t_2.st_id
    where money>0
    group by test_grp)
    as t2 using(test_grp)
    )
as tabel_3 
on tabel_1.test_grp=tabel_3.test_grp
left join 
    (
    select test_grp,
       id_activ_money*100/cnt_id_active as CR_activ   -- count of buyers from active users/ count of avtive users
    from (
    select test_grp,
        count(id) as cnt_id_active
    from
        (select 
            st_id as id,
            sum(correct) as sum_cor
        from default.peas
        group by st_id
        having sum_cor>10
        ) as a
    inner join 
        (
        select 
            test_grp,
            st_id
        from default.studs
        ) as b
    on a.id=b.st_id
    group by test_grp
        ) as t1
    left join
    (
        select test_grp,
    count(money) as id_activ_money
    from
        (select 
            st_id as id,
            sum(correct) as sum_cor
        from default.peas
        group by st_id
        having sum_cor>10
        ) as a
    inner join 
        (
        select 
            test_grp,
            st_id
        from default.studs
        ) as b
    on a.id=b.st_id
    inner join 
        (select 
            st_id,
            money
        from default.final_project_check
        where money>0) as c
    on  b.st_id=c.st_id
    group by test_grp
    )
    as t2 using (test_grp)
    )
as tabel_4
on tabel_1.test_grp=tabel_4.test_grp
left join 
    (
select test_grp,
        sum(math_money)/count(subj_corr)*100 as CR_math --- count of buyers from math active users/count math active users  
        from 
            (select  a.st_id as st_id,                  
                     a.test_grp as test_grp,
                     b.subject as subj_corr,
                     c.math_money as math_money
            from (select distinct st_id, subject, SUM(correct) as corr     
                         from default.peas                                   
                         group by st_id,subject,correct
                         having subject=='Math' and corr >= 2) as b
              left join default.studs as a
              on b.st_id=a.st_id
              left join (select st_id, subject, 
                        case                            
                        when subject=='Math'then 1
                        else 0
                        end as math_money
                        from default.final_project_check 
                        group by st_id, subject, money) as c
            on b.st_id=c.st_id)
            group by test_grp
    )
as tabel_5
on tabel_1.test_grp=tabel_5.test_grp
    '''

q_test = ph.read_clickhouse(query=qqq, connection=connection_default)
q_test

,group_,ARPU,ARPAU,CR,CR_activ,CR_math
0,control,4540.98,10393.70,4.92,13.39,6.0
1,pilot,11508.47,29739.58,10.85,36.46,8.7
